# Reviews preprocessing pipeline

This notebook takes a collection of reviews, collects the text contnents, and cleans/sorts the resulting words ina dictionary.

Request url with any url.  Replace starts ans stops w/ corresponding first/last words.

In [1]:
# Pulls a series of urls for analysis.  Declutters with start/stop sections.
my_urls = ["https://www.eurogamer.net/articles/2021-09-10-in-defence-of-god-of-wars-thor","https://playcrazygame.com/2021/09/13/ragnarok-explain-the-god-thors-look/","https://codelist.biz/2021/09/13/thor-is-just-as-thick-as-in-avengers-endgame/"]

starts = ["Last ", "The revelation","The secret"]
stops = ["bad joke.", "without a defined date.", "but also new allies."]

In [ ]:
# This cell bwgins to clean and organize the urls.  It gets around some scrape blocking, operates within the start/stop margins, and removes unwanted characters.
def collect_reviews(my_urls):
    contents = ""
    loc = 0

    from urllib.request import urlopen, Request
 
    for url in my_urls:
        req = Request(url)
        req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:82.0) Gecko/20100101 Firefox/82.0')
        webContents = urlopen(req).read().decode("latin-1")
        webContents = str(webContents)
        
        cleaned = clean_reviews(webContents, starts[loc], stops[loc])
        contents += str(cleaned)
        loc += 1

        contents = contents.replace("\\n","")
        contents = contents.replace("\\r","")
        contents = contents.replace("\\t","")
        
        
    return contents

In [ ]:
# Defines a function to clean the reviews and scrubs the html.
def clean_reviews(contents, start, end):
    startLoc = contents.find(start)
    endLoc = contents.find(end)
    contents = contents[startLoc: endLoc]

    inside = 0
    text = ''

    for char in contents:
        if char == '<' or char == '{':
            inside = 1
        elif (inside == 1 and char == '>' or char == '}'):
            inside = 0
        elif inside== 1:
            continue
        else: 
            text += char

    return[text]

In [ ]:
# Defines a function to convert to lowercase,  normalize text with re, and return a word bag without the mes.
def normalize_text(text):
    text = text.lower()
    import re
    word_bag = re.compile(r'\W+', re.UNICODE).split(text)
    return(word_bag)

In [ ]:
# Defines a funtion to create a dictionary from the above word bag, zips together words and their frequwncy.
def wordsToDictionary(word_bag):
    word_freq = [word_bag.count(word) for word in word_bag]
    return dict(list(zip(word_bag,word_freq)))


In [ ]:
# sorts words by freq, most to least common
def sortDictionary(counted_words):
    aux = [(counted_words[key], key) for key in counted_words]
    aux.sort()
    aux.reverse()
    return aux

In [ ]:
# Creates a list of stop words, then defines func to reomve them from word bag
stopwords = ['a','n','the','my','for','with','login','facebook','functionalityfunction','src','id','parentnode','if','is','was','navigation','main','ul','menu','nav']
def removeStopWords(word_bag):
    return [w for w in word_bag if w not in stopwords]

In [ ]:
collected_text = collect_reviews(my_urls)
collected_words = normalize_text(collected_text)
collected_words = removeStopWords(collected_words)
dict_words = sortDictionary(wordsToDictionary(collected_words))

print(dict_words)